In [9]:
!unzip to\ isha.zip

Archive:  to isha.zip
   creating: to isha/already resized images/
  inflating: to isha/already resized images/1_resized.png  
  inflating: to isha/already resized images/2_resized.png  
  inflating: to isha/already resized images/3_resized.png  
  inflating: to isha/already resized images/4_resized.png  
  inflating: to isha/already resized images/5_resized.png  
  inflating: to isha/already resized images/5b0d3ee884d57c0004cba658_2020-03-19_22-40-52-154998 resized resized resized resized.jpg  
  inflating: to isha/already resized images/5b0d3ee884d57c0004cba658_2020-04-23_14-01-34-858771 resized.jpg  
  inflating: to isha/already resized images/5b0d3ee884d57c0004cba658_2020-04-30_19-01-51-932662 resized.jpg  
  inflating: to isha/already resized images/5b0d3f7284d57c0004cba6e5_2020-03-19_22-40-52-864341 resized resized resized.jpg  
  inflating: to isha/already resized images/5b0d3f7284d57c0004cba6e5_2020-05-17_10-02-40-358072 resized.jpg  
  inflating: to isha/already resized images

In [12]:
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import xml.etree.ElementTree as ET
import json 

def jsonify(root):
    parsed = dict()
    for child in root:
        children = child.getchildren()
        if len(children) == 0:
            parsed[f"{child.tag}"] = [child.text]
        else:
            if f"{child.tag}" in parsed.keys():
                parsed[f"{child.tag}"].append(jsonify(children))
            else:
                parsed[f"{child.tag}"] = [jsonify(children)]
    for item in parsed.keys():
        if len(parsed[item]) == 1:
            parsed[item] = parsed[item][0]
    return parsed

def xml_json(filename):
    tree = ET.parse(filename)
    root = tree.getroot()
    return jsonify(root)

class box():
    def __init__(self, box_dict):
        self.xmin = int(box_dict["xmin"])
        self.xmax = int(box_dict["xmax"])
        self.ymin = int(box_dict["ymin"])
        self.ymax = int(box_dict["ymax"])
        self.x = False
        pass
    
    @property
    def width(self):
        return self.xmax - self.xmin
    
    @property
    def height(self):
        return self.ymax - self.ymin
    
    @property
    def center(self):
        return (int((self.xmax + self.xmin)/2), int((self.ymax + self.ymin)/2))
    
    @property
    def area(self):
        return self.width * self.height
    
    # @property
    # def x(self):
    #     return self.__x

    def set_x(self, close):
        if not self.x:
            self.x = close
        return
    
    def __sub__(self, new):
        i = np.array(list(self.center))
        j = np.array(list(new.center))
        return np.sqrt(np.sum(np.square(i-j)))

    def patch(self, ax = None):
        if self.x:
            color = 'red'
        else:
            color = "green"

        rect = patches.Rectangle((self.xmin, self.ymin), self.xmax - self.xmin, self.ymax - self.ymin, linewidth=1,edgecolor=color, facecolor='none')
        center = patches.Circle(self.center, radius = 5, edgecolor = color, facecolor = color)
        if ax != None:
            ax.add_patch(rect)
            ax.add_patch(center)
        return rect, center

    def __str__(self):
        return f"(w,h), {self.width}, {self.height}\ncenter:{self.center}\narea:{self.area}"
    
    def __repr__(self):
        return str(self)

def get_filenames(folder = "to isha"):
    file_names = []
    for dirname, directs, files in os.walk(folder):
        for file in files: 
            name = f"{dirname}/{file}"
            file_names.append(name)
    return (file_names)

def get_raw_files(files):
    jsons = []
    images = dict()
    for name in files:
        if ".xml" in name:
            jsons.append(xml_json(name))
        elif ".png" or ".jpg" or ".jpeg" in name:
            images[name.split("/")[-1]] = np.array(cv2.imread(name))
    return jsons, images

def too_close(a, b, threshold = 1):
    prespective_similarity = min(a.area/b.area, b.area/a.area) # size of box
    inverse_relative_distance = np.sqrt((max(a.area, b.area)))/(a - b)
    state = prespective_similarity * inverse_relative_distance
    return state > threshold

def too_close(a, b, threshold = 1):
    prespective_similarity = min(a.area/b.area, b.area/a.area) # size of box
    inverse_relative_distance = (max(a.area, b.area))/((a - b)**2)
    state = prespective_similarity * inverse_relative_distance
    return state > threshold

def validate_image(image, json_file, show_image = False, path = True, thresh = 0.75):
    if path: 
        image = plt.imread(image)
        im_det = json.loads(open(json_file,"r").read())
    else:
        im_det = json_file

    boxs = []
    done = set()

    if "object" in im_det.keys():
        if type(im_det["object"]) == list:
            for bo in im_det["object"]:
                b = box(bo["bndbox"])
                boxs.append(b)
        else:
            bo = im_det["object"]
            b = box(bo["bndbox"])
            boxs.append(b)
    
    for b in boxs:
        for c in boxs:
            current = [str(b),str(c)]
            current.sort()
            key = str(hash(str(current)))
            if b != c and key not in done:
                done.update({key})
                close = too_close(b,c, threshold = thresh)
                b.set_x(close)
                c.set_x(close)
    
    violations = 0
    for b in boxs:
       violations += int(b.x)

    if show_image: 
        fig, ax = plt.subplots(1)
        fig.set_size_inches(10.5, 5.5)
        for b in boxs:
            b.patch(ax)
        ax.imshow(image)
        plt.show()
    return violations


In [13]:

# path to json and path to image code
# ----------------------------------
image_path = "/content/2_resized.png"
json_path = "/content/2_resized_visual.json"
violations = validate_image(image_path, json_path, show_image = False, path = True, thresh = 0.75)
print(f"violations: {violations}")

# xml code
# --------
# files = get_filenames(folder = "to isha")
# jsons, images = get_raw_files(files)
# for im_num in range(len(jsons)):
#     image_path = images[jsons[im_num]["filename"]]
#     json_path = jsons[im_num]
#     violations = validate_image(image_path, json_path, show_image = True, path = False)
#     print(violations)

violations: 64
